In [ ]:
len(
    "
    
    Shree Bhavan Pure South Indian Vegetarian Restaurant, 641 among Bangkok restaurants: 765 reviews by visitors and 33 detailed photos. Find on the map and call to book a table. ... Very good experience and dining Service: Dine in Meal type: Dinner Price per person: ฿200-300 Food: 5 Service: 5 Atmosphere: 5 Recommended dishes: ... The cultural significance of street food here is something to appreciate; it's a reflection of Bangkok's rich culinary heritage and the vibrant spirit of sharing meals that brings people together. 120 Soi 19 Phetchaburi Rd, Thanon Phaya Thai, Ratchathewi, Bangkok 10400, Thailand. 10. Sanjha Chulha Pratunam Bangkok. Saravana Bhavan is part of an international chain that serves up South Indian vegetarian dishes, and this Bangkok outpost does not disappoint. Located in Baan Silom, the Indian restaurant definitely has one of the largest vegetarian menus (over 100 dishes) in Bangkok, praised for their "rich and buttery" flavours. Sarvana Bhavan is one of the best South Indian vegetarian restaurants in Bangkok. The food consists of authentic lentils and rice with flavourful spices. You can find many Tamilians in this restaurant. Dosa, Uthapam, Onion Uttapam, and Masala Dosa are the dishes you can enjoy there that won't give you a missed-out feeling away from your homeland. Maa Pure Vegetarian Indian Restaurant, #1007 among Bangkok restaurants: 472 reviews by visitors and 50 detailed photos. ... Service: Dine in Meal type: Dinner Price per person: ฿800-900 Food: 5 Service: 3 Atmosphere: ... Shree Khodiyar Kathiyawadi Dhaba pure vegetarian Indian restaurant Bangkok
    "
)